In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px 
import seaborn as sns 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from math import radians, cos, sin, asin, sqrt
def calculate_distance(lat1, lat2, lon1, lon2):
	# The math module contains a function named
	# radians which converts from degrees to radians.
	lon1 = radians(lon1)
	lon2 = radians(lon2)
	lat1 = radians(lat1)
	lat2 = radians(lat2)
	
	# Haversine formula
	dlon = lon2 - lon1
	dlat = lat2 - lat1
	a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2

	c = 2 * asin(sqrt(a))
	
	# Radius of earth in kilometers. Use 3956 for miles
	r = 6371
	
	# calculate the result
	return(c * r)

In [3]:
data = pd.read_csv('../input/uber-fares-dataset/uber.csv')

In [15]:
data.columns

In [16]:
data.info()

In [ ]:
#type casting : 
data

In [7]:
#fix the weirdness of ccoordinate
data.describe()
data.dtypes
print('Null Values :')
data.isnull().sum()

In [8]:
data = data.drop_duplicates(keep='first')
data

In [9]:
data_filtered = data.loc[(((data['pickup_longitude'] <= 180) & (data['pickup_longitude'] >= -180)) & (data['dropoff_longitude'] <= 180)) & (data['dropoff_longitude'] >= -180)]
data_filtered = data_filtered.loc[(((data_filtered['pickup_latitude'] <= 90) & (data_filtered['pickup_latitude'] >= -90)) & (data_filtered['dropoff_latitude'] <= 180)) & (data_filtered['dropoff_latitude'] >= -90)]
data_filtered

## MARKDOWN

In [14]:
example = data_filtered.iloc[:10]
def print_(data) : 
    print(data.shape)
example.apply(lambda x : print_(example))


In [24]:
def remove_outliers(data:pd.DataFrame,exeption_col = ['Unnamed: 0','key','pickup_datetime','pickup_longitude','pickup_latitude',
                                                    'dropoff_longitude','dropoff_latitude'])->pd.DataFrame : 
    """
    a function to remove outlier within pandas dataframe with iqr method
    Args : 
    data : pandas dataframe
    Return
    pd.DataFrame(cleaned)
    """
    for col in data.select_dtypes(exclude='object').columns : 
        if col not in exeption_col : 
#creating function to remove outlier by interquartil range 
            q1_ = np.quantile(data[col],0.25)
            q3_ = np.quantile(data[col],0.75)
            iqr_ = q3_ - q1_
            # Create the lower & upper bound 
            lower_ = q1_ - 1.5*iqr_
            upper_ = q3_ + 1.5*iqr_
            #implement using joblib 
            for val in data[col].values : 
                if lower_ <= val <= upper_:
                    # Do nothing when the column value inside the safe boundary
                    continue
                else:
                    #drop the values
                    filter_drop = data[data[col] == val].index
                    data.drop(filter_drop,axis=0,inplace=True)
                    data.reindex()
        else : 
            continue
        
    print('Done')
    return data


In [25]:
data_removed_outliers = remove_outliers(data=data_filtered)

In [26]:
data_removed_outliers.to_csv('data_removed_outliers.csv',index=False)

In [34]:
#create reading checkpoint to avoid time consuming process of removing outlier 
data_removed_outliers = pd.read_csv('data_removed_outliers.csv')

In [35]:
data_removed_outliers.shape

In [ ]:
lat1, lat2, lon1, lon2

In [29]:
data_removed_outliers.columns

In [36]:
data_removed_outliers['distance'] = data_removed_outliers.apply(lambda x : calculate_distance(x['pickup_latitude'], x['dropoff_latitude'], 
                                                                                  x['pickup_longitude'],x['dropoff_longitude']),axis=1)
#     lat1=data_removed_outliers['pickup_latitude'],
#                                                       lat2=data_removed_outliers['dropoff_latitude'],
#                                                       lon1=data_removed_outliers['pickup_longitude'],
#                                                       lon2=data_removed_outliers['dropoff_longitude'])


In [37]:
data_removed_outliers

In [38]:
data_removed_outliers['pickup_datetime'] = pd.to_datetime(data_removed_outliers['pickup_datetime'], infer_datetime_format=True)
data_removed_outliers['pickup_datetime_year'] = data_removed_outliers['pickup_datetime'].dt.year
data_removed_outliers['pickup_datetime_dayofmonth'] = data_removed_outliers['pickup_datetime'].dt.day
data_removed_outliers['pickup_datetime_dayofweek'] = data_removed_outliers['pickup_datetime'].dt.dayofweek
data_removed_outliers['pickup_datetime_hour'] = data_removed_outliers['pickup_datetime'].dt.hour
data_removed_outliers['pickup_datetime_month_name'] = data_removed_outliers['pickup_datetime'].dt.month_name()
data_removed_outliers

In [39]:
data_removed_outliers.to_csv('ready_for_EDA.csv',index=False)

In [40]:
data_eda = pd.read_csv('ready_for_EDA.csv')

In [ ]:
#TODO : 
# Try to discover pattern on each day a week is weekday or sunday 
# try to discover pattern month 
#try to discover pattern in fare 
# try to discover busy hour (heatmap) extracting datetime
# try to discover density in area -> classify into dense / non dense area (try to look for a dynamic pricing paper)